# Build a Seldon-core Microservice
> Get an ML endpoint up and running!

- toc: true 
- badges: true
- comments: true
- categories: [seldon-core, docker]

### Reqs
* docker
    * If you are coming from [Launch a local kubernetes cluster](https://ntorba.github.io/writing/jupyter/2020/07/17/local-kubernetes.html), you are good to follow this example. If not, you can quickly follow that post before running the example here!

### Goal
* Prepare a docker image with seldon-core for deployment on kubernetes

#### Steps
1. Define a seldon python component
2. Build docker image
3. Run a container to test code

### Define Python Component
I'm taking this example code directly from [seldon-core irisClassifier example](https://github.com/SeldonIO/seldon-core/blob/master/examples/models/sklearn_iris/sklearn_iris.ipynb). 
First, we train a model based on the iris dataset included in the sklearn package, then we serve that trained model in the seldon endpoint.

In [4]:
#hide_output
!mkdir iris_classifier

mkdir: iris_classifier: File exists


In [1]:
%%writefile iris_classifier/train_iris.py
#collapse_show
#hide_output
import joblib
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import datasets


OUTPUT_FILE = "iris_classifier/IrisClassifier.sav"


print("Loading iris data set...")
iris = datasets.load_iris()
X, y = iris.data, iris.target
print("Dataset loaded!")

clf = LogisticRegression(solver="liblinear", multi_class="ovr")
p = Pipeline([("clf", clf)])
print("Training model...")
p.fit(X, y)
print("Model trained!")

print(f"Saving model in {OUTPUT_FILE}")
joblib.dump(p, OUTPUT_FILE)
print("Model saved!")




Overwriting iris_classifier/train_iris.py


In [6]:
#hide_output
!python iris_classifier/train_iris.py

Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in iris_classifier/IrisClassifier.sav
Model saved!


Next, we define the seldon python component that will be used to serve the model. 
Seldon has a few [components](https://docs.seldon.io/projects/seldon-core/en/v1.1.0/python/python_component.html). In this example, we only use the Model component. Seldon components hold the logic that will be implanted into the serving endpoint that seldon creates. The model component must have a predict function, which is called when the future endpoint is hit. 
The reason seldon is so useful is because this is the only python code we need to write to serve this model. Seldon provides the rest of the logic, which puts this component into a web server, to serve the model.

An important note about this section is that you'lll see the file is named `IrisClassifier.py`, which is camelcased. This is important, and you should not change this. The file name and the python component class name **must match**. 

In [7]:
%%writefile iris_classifier/IrisClassifier.py
#collapse_show
#hide_output
import joblib

class IrisClassifier(object):

    def __init__(self):
        self.model = joblib.load('IrisClassifier.sav')

    def predict(self,X,features_names):
        return self.model.predict_proba(X)

Overwriting iris_classifier/IrisClassifier.py


### Build Docker Image
After defining a python component, there are two ways to create the docker image necessary for deployment. 
* [define a Dockerfile](https://docs.seldon.io/projects/seldon-core/en/v1.1.0/python/python_wrapping_docker.html) which launches the seldon microservice
* use [s2i](https://docs.seldon.io/projects/seldon-core/en/v1.1.0/wrappers/s2i.html) to build the image directly from source code. 

I prefer manually defining a Dockerfile because it provides more control over the process. However, s2i is a great tool that works just as well. 



#### Write requirements.txt 
We must write a requirements.txt library with all requirements for the docker image listed.

In [8]:
%%writefile iris_classifier/requirements.txt
#hide_output
sklearn
seldon-core

Overwriting iris_classifier/requirements.txt


#### Define Dockerfile
The Dockerfile follows the example provided [here](https://docs.seldon.io/projects/seldon-core/en/v1.1.0/python/python_wrapping_docker.html). 
We start from the python:3.7-slim base image, copy the code from the current directory, which includes the python component we defined earlier, install requirements, then expose port 5000 for the microservice to run. 
Next, we define seldon specific variables. 
* MODEL_NAME must match the python file name (which also much match the python component class name). 
* API_TYPE can be either REST or GRPC.
* SERVICE_TYPE is the type of seldon component. MODEL for this example. (explore the other seldon components [here]()
* PERSISTENCE: 0 or 1. Defaults to 0. If it is set to 1, the component class will be periodically persisted to reis. This s unnecessary for our case because the component class will not change.
    * this would be more pertinent for components like routers, which can have updating states for long running jobs. 

In [50]:
%%writefile iris_classifier/Dockerfile
#collapse_show
#hide_output
FROM python:3.7-slim
COPY . /app
WORKDIR /app
RUN pip install -r requirements.txt
EXPOSE 5000

# Define environment variable
ENV MODEL_NAME IrisClassifier 
ENV API_TYPE REST
ENV SERVICE_TYPE MODEL 
ENV PERSISTENCE 0

# seldon-core-microservice is a command line tool installed with the seldon-core python libray. You can use this locally as well!
CMD exec seldon-core-microservice $MODEL_NAME $API_TYPE --service-type $SERVICE_TYPE --persistence $PERSISTENCE

Overwriting iris_classifier/Dockerfile


To test this example, let's build and run the docker image! 

#### Docker Build
Pass the iris_classifier dir where the image guts live, then pass a -t to tag the image with a name referring to your preferred docker image repository (I'm running on locally).

In [51]:
#hide_output
!docker build iris_classifier/ -t localhost:5000/iris_ex:latest

Sending build context to Docker daemon  11.78kB
Step 1/10 : FROM python:3.7-slim
 ---> b386e7420fc3
Step 2/10 : COPY . /app
 ---> 4ff1fc2d09e5
Step 3/10 : WORKDIR /app
 ---> Running in 0e5b783b9df2
Removing intermediate container 0e5b783b9df2
 ---> 840bd996fe26
Step 4/10 : RUN pip install -r requirements.txt
 ---> Running in 6f1af0205271
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=f35bba8878dbd98e914c5a5adaf27a0f1d876a5edecb8bc12a6fc820567cb767
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
  Created wheel for Flask-OpenTracing: filename=Flask_OpenTracing-1.1.0-py3-none-any.whl size=9070 sha256=a5523f52ee6f8ac6d14676c4ae4366329ed5d0de51efad438d62490954d8b0a0
  Stored in directory: /root/.cache/pip/wheels/42/22/cd/ccb93fa68f4a01fb6c10082f97bcb2af9eb8e43565ce38a292
  Created wheel for opentracing: filename=opentracing-2.3.0-py3-none-any.whl size=51347 sha256=af662d9bfa99e590a3ea577797b

### Test Image
You can test your newly created image by running the image and hitting the endpoint. 
You may ask yourself at this point, "if I have a working docker image, what do I need kubernetes for?" 
This is a great question. For simple use cases, this docker image itself is all you need, and you could run it as a standalone service. If the load is small and you can run it without any load balancing functionalities, you are good to go. 
However, kubernetes is a container orchestration engine. That means it is built to handle complex containerized applications and will make your life much easier if you need to handle more complex operations for applications that need to serve on a large scale. 

In [52]:
#hide_output
!docker run --name "iris_predictor" -d --rm -p 5001:5000 localhost:5000/iris_ex:latest

4d88f1163a71622fc2b67f33b8af4e95c2c8dafa9da43e2fe8c06e4322b7591c


You could also remove the -d argument from the above command and run this command in a separate window to see the log output while sending requests to the endpoint. Test the endpoint with the curl below! 

In [53]:
import numpy as np
import grpc 
from seldon_core.proto import prediction_pb2
from seldon_core.proto import prediction_pb2_grpc


### Test Rest Endpoint
!curl -s http://localhost:5001/predict -H "Content-Type: application/json" -d '{"data":{"ndarray":[[5.964,4.006,2.081,1.031]]}}'


### Test GRPC Endpoint
# data = np.array([[5.964,4.006,2.081,1.031]])

# datadef = prediction_pb2.DefaultData(
#     tensor=prediction_pb2.Tensor(shape=data.shape, values=data.flatten())
# )
# request = prediction_pb2.SeldonMessage(data=datadef)
# with grpc.insecure_channel("localhost:5001") as channel:
#     stub = prediction_pb2_grpc.ModelStub(channel)
#     response = stub.Predict(request=request)
# print(response)

If you see successful output, you have your first seldon-core microservice up and running, congrats! 
Use the next command to stop the running docker container: 

In [ ]:
!docker container rm iris_predictor --force

If you are like me, you are a bit surprised at just how easy it was to create this. That is the power of seldon. 

You may be wondering, what about kubernetes? Didn't your first post in this series say that you must have access to a cluster? That is still correct, and you will need that cluster in the following posts. I just like to start with this piece because it is straightforward and a rewarding way to start. 

At this stage you are prepared to move onto any of these posts: 
* [Deploy First Microservice]()
    * This post is for those excited to deploy your new docker image on kubernetes! 
* [Build Various Seldon-core microservices]()
    * This post is for those more interested in learning about the other seldon components beside models

### STOP HERE
* Once the docker image is created, I will create a new post that is specific to launching on kubernetes
* Getting a working docker image is half the battle, and this can be a big success for users
    * this is also a great jumping off point to instead say, want to make a more complex inference graph before kubernetes? or want to go learn about other python components before moving on to kubernetes?
* 1) building docker, 2) launch on kubernetes
    * in the light of mouldarity, I am better off keeping these steps separate from each other so users can focus on the docker and seldon stuff if they like
    * this has the other advantage that if users struggle with getting a local k8s cluster up, they can still leaving the tutorials feelings they have gained a good understanding of seldon because they have their nice docker images they built

First, let's take down the running docker container: